<a href="https://colab.research.google.com/github/HariboNinja/First_Website/blob/main/Final_Spotify_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools
import datetime
import time
import base64
import requests
from time import sleep

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from requests import post, get
import matplotlib.pyplot as plt

# import spotipy
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth

import warnings
warnings.filterwarnings("ignore")

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client_id = "8045771a68234001bf3d838d2f584842"
client_secret = "4c53de9b45ac4fd3bad2629860ccd54a"
%env SPOTIPY_CLIENT_ID="8045771a68234001bf3d838d2f584842"
%env SPOTIPY_CLIENT_SECRET="4c53de9b45ac4fd3bad2629860ccd54a"
%env SPOTIPY_REDIRECT_URI="https://developer.spotify.com"
print(client_id)
print(client_secret)

env: SPOTIPY_CLIENT_ID="8045771a68234001bf3d838d2f584842"
env: SPOTIPY_CLIENT_SECRET="4c53de9b45ac4fd3bad2629860ccd54a"
env: SPOTIPY_REDIRECT_URI="https://developer.spotify.com"
8045771a68234001bf3d838d2f584842
4c53de9b45ac4fd3bad2629860ccd54a


In [ ]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def get_token():
  auth_string = client_id + ":" + client_secret
  auth_bytes = auth_string.encode("utf-8")
  auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
  url = "https://accounts.spotify.com/api/token"

  headers = {
      "Authorization": "Basic " + auth_base64,
      "Content-Type": "application/x-www-form-urlencoded"
  }

  data = {"grant_type": "client_credentials"}

  result = requests.post(url, headers=headers, data=data)
  json_result = json.loads(result.content)
  token = json_result["access_token"]
  return token

def get_auth_header(token):
  return {"Authorization": "Bearer " + token}


def search_for_artist(token, artist):
  url = "https://api.spotify.com/v1/search"
  headers = get_auth_header(token)
  query = f"?q={artist}&type=artist&limit=50"

  query_url = url + query
  result = requests.get(query_url, headers=headers)
  json_result = json.loads(result.content)["artists"]["items"]
  if len(json_result) == 0:
    print("No artist was found with this name")
    return None

  return json_result

def get_songs_by_artist(token, artist_id):
    url: str = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

def get_audio_features(token, track_id):
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)
    return json_result

def search_for_song(token, song):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={song}&type=track&limit=1"
    query_url = url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)["tracks"]["items"]
    if len(json_result) == 0:
        print("No song was found with this name")
        return None
    return json_result[0]


# if __name__ == "__main__":

#   token = get_token()
#   auth_header = get_auth_header(token)
#   print(auth_header)

#   get_token()

#   print(token)

In [ ]:
token = get_token()
results = search_for_artist(token, "Coldplay")
artist_id = results[0]['id']
print(artist_id)



4gzpq5DPGxSnKTe4SA8HAU


In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]


In [ ]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    just_uri = track_uri.split(":")[-1]


    #Track name
    track_name = track["track"]["name"]

    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)

    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]

    #Album
    album = track["track"]["album"]["name"]

    #Popularity of the track
    track_pop = track["track"]["popularity"]
    print(just_uri)

2plbrEY59IikOBgBGLjaoe
6dOtVTDdiauQNBQEDOtlAB
7tI8dRuH2Yc6RuoTjxo4dU
5G2f63n7IPVPPjfNIGih7Q
2HRqTpkrJO5ggZyyK6NPWz
0WbMK4wrZ1wFSty9F7FCgu
1Es7AUAhQvapIcoh3qMKDL
6WatFBLVB0x077xWeoVc2k
2tHwzyyOLoWSFqYNjeVMzj
3xkHsmpQCBMytMJNiDf3Ii
2262bWmqomIaJXwCRHr13j
1UHS8Rf6h5Ar3CDWRd3wjF
3QaPy1KgI7nu9FJEQUgn6h
0Sr7ssScx54yxdM2ols7nW
5fZJQrFKWQLb7FpJXZ1g7K
2QjOHCTQ1Jl3zawyYOpxh6
2PnlsTsOTLE5jnBnNe2K0A
76JKIsdKrAfWUMjaA0u7v5
51rfRCiUSvxXlCSCfIztBy
7CyPwkp0oE8Ro9Dd5CUDjW
5XeFesFbtLpXzIVDNQP22n
3yDRcs0Y4pPzkvMbUfeF9H
17phhZDn6oGtzMe56NuWvj
42VsgItocQwOQC3XWZ8JNA
6X6m4xmdFcz31p1h7Qg68c
42wdz3j7gstgUuhoFLCXqd
5IZXB5IKAD2qlvTPJYDCFB
0io16MKpbeDIdYzmGpQaES
2esZG2XFtuoWWA9AfDvSxy
0IsIY8pfu1yaGkPUD7pkDx
3qhlB30KknSejmIvZZLjOD
51ZQ1vr10ffzbwIjDCwqm4
6MzofobZt2dm0Kf1hTThFz
4xdBrk0nFZaP54vvZj0yx7
7KA4W4McWYRpgf0fWsJZWB
2aYZaN5SmkRDLsrrV8GkBQ
3hRV0jL3vUpRrcy398teAU
5lKFO0PrOC9jmOlPEdjHsL
2HRgqmZQC0MC7GeNuDIXHN
7bywjHOc0wSjGGbj04XbVi
6PhRhocBXP4y7NpciGmzra
3JG1uFc40wfyrqaWC7iv0e
6AI3ezQ4o3HUoP6Dhudph3
5AJ9hqTS2wc

In [ ]:
billboard = pd.read_csv('/content/drive/MyDrive/billboard_24years_lyrics_spotify.csv')
billboard['track_id'] = billboard['uri'].str.replace('spotify:track:', '')
bill_list = list(billboard['track_id'])

In [ ]:
tracks_2_playlist = list(billboard["uri"])

In [ ]:
tracks_2_playlist

['spotify:track:3y4LxiYMgDl4RethdzpmNe',
 'spotify:track:0n2SEXB2qoRQg171q7XqeW',
 'spotify:track:5IALWUYK0zDSEmZgb4ICvc',
 'spotify:track:3XKIUb7HzIF1Vu9usunMzc',
 'spotify:track:3XKIUb7HzIF1Vu9usunMzc',
 'spotify:track:1m2xMsxbtxv21Brome189p',
 'spotify:track:4cKGldbhGJniI8BrB3K6tb',
 'spotify:track:7H6ev70Weq6DdpZyyTmUXk',
 'spotify:track:6nozDLxeL0TE4MS9GqYU1v',
 'spotify:track:6qc34bnVOyqGDPni8H5W0U',
 'spotify:track:6fA7akEuTUL3dW1V0GELaZ',
 'spotify:track:7f1Dmr246cJ9uQYdbplTbh',
 'spotify:track:1ZozJfi8u9cO2Ob8KwiwNT',
 'spotify:track:6Wnlq1Pu3I0BajLvxn05Vr',
 'spotify:track:4pmc2AxSEq6g7hPVlJCPyP',
 'spotify:track:5Mmk2ii6laakqfeCT7OnVD',
 'spotify:track:6ZOBP3NvffbU4SZcrnt1k6',
 'spotify:track:3IQMlYFrnoTa6xJ15rlqTN',
 'spotify:track:2BcOZ4vOeu2qZowIHrvTYy',
 'spotify:track:0ekdT3jOhn3LinWtqHhmsw',
 'spotify:track:5bGmuxShUba9maPswDnhCs',
 'spotify:track:6mwA6YiKDjAUG8kWvRRUPh',
 'spotify:track:62bOmKYxYg7dhrC6gH9vFn',
 'spotify:track:2dwhMQsFeHr2S787WxqAqW',
 'spotify:track:

In [ ]:
bill_list

['3y4LxiYMgDl4RethdzpmNe',
 '0n2SEXB2qoRQg171q7XqeW',
 '5IALWUYK0zDSEmZgb4ICvc',
 '3XKIUb7HzIF1Vu9usunMzc',
 '3XKIUb7HzIF1Vu9usunMzc',
 '1m2xMsxbtxv21Brome189p',
 '4cKGldbhGJniI8BrB3K6tb',
 '7H6ev70Weq6DdpZyyTmUXk',
 '6nozDLxeL0TE4MS9GqYU1v',
 '6qc34bnVOyqGDPni8H5W0U',
 '6fA7akEuTUL3dW1V0GELaZ',
 '7f1Dmr246cJ9uQYdbplTbh',
 '1ZozJfi8u9cO2Ob8KwiwNT',
 '6Wnlq1Pu3I0BajLvxn05Vr',
 '4pmc2AxSEq6g7hPVlJCPyP',
 '5Mmk2ii6laakqfeCT7OnVD',
 '6ZOBP3NvffbU4SZcrnt1k6',
 '3IQMlYFrnoTa6xJ15rlqTN',
 '2BcOZ4vOeu2qZowIHrvTYy',
 '0ekdT3jOhn3LinWtqHhmsw',
 '5bGmuxShUba9maPswDnhCs',
 '6mwA6YiKDjAUG8kWvRRUPh',
 '62bOmKYxYg7dhrC6gH9vFn',
 '2dwhMQsFeHr2S787WxqAqW',
 '7ffwRz8lZyDOE4Vj58Lo72',
 '0AcLrSfAEBQcUnHOTm5pXg',
 '3w8Mw9GHYepoTWOSdiyosj',
 '5TAf4lnZCZTLlZHNZMLFLi',
 '2n3Ot3QDWJBI9itZEiBhYl',
 '5s4catxeZsaWFnOrvrXZHf',
 '3wMUvT6eIw2L5cZFG1yH9j',
 '1cjBan0t4eBk2Y5j17hdyf',
 '3BsaRV5QIulYz2lV9WWa8T',
 '7mYvtEeBdMqRSyj1Qpv6my',
 '2oPxlNO0xKUddhNm3Vc4Lq',
 '6tdiCcsSVUQ7d7AR1wM6IQ',
 '1TyyvOlHGyEUWHr5WTlnzH',
 

In [ ]:
infooo = []
audio_features_list=[]
audio_genre_list=[]

In [ ]:
new_list = []

for item in infooo:
    new_list.append(item[0])

pd.DataFrame(new_list)


""


In [ ]:
for i in range(0, len(bill_list), 100):
  batch = bill_list[i:i + 100]

  audio_features_results = sp.audio_features(batch)
  audio_features_list.extend(audio_features_results)

  sleep(30)
  print(i)


KeyboardInterrupt: 

In [ ]:
audio_df = pd.DataFrame(audio_features_list)

audio_df

In [ ]:
audio_df.to_csv('Billboard_0.csv', index=False)

In [ ]:
len(audio_df)

In [ ]:
for i in range(0, len(bill_list), 100):
  batch = bill_list[i:i + 100]

  audio_genre_results = sp.recommendation_genre_seeds()
  audio_genre_list.extend(audio_genre_results)

  sleep(30)
  print(i)
https://open.spotify.com/playlist/7hmtgny8yFXyg2FmCmC75r?si=gjEgM0YbR8W7-T0UVaUEOA

0
100


KeyboardInterrupt: 

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [ ]:
playlist_id = "7hmtgny8yFXyg2FmCmC75r"
sp.playlist_add_items(playlist_id, tracks_2_playlist)


Enter the URL you were redirected to: https://open.spotify.com/playlist/7hmtgny8yFXyg2FmCmC75r?si=AQBQCtn1TC2JmaZJq_jzgw


SpotifyOauthError: error: invalid_request, error_description: code must be supplied

In [ ]:
# recommendation_genre_seeds()

In [ ]:
test = sp.track(bill_list[1])

In [ ]:
test["uri"]

'spotify:track:3y4LxiYMgDl4RethdzpmNe'

In [ ]:
sp.album_tracks(album_id)

{'href': 'https://api.spotify.com/v1/albums/10aiDpdFGyfCFEcqpx6XTq/tracks?offset=0&limit=50',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6GI52t8N5F02MxU0g5U69P'},
     'href': 'https://api.spotify.com/v1/artists/6GI52t8N5F02MxU0g5U69P',
     'id': '6GI52t8N5F02MxU0g5U69P',
     'name': 'Santana',
     'type': 'artist',
     'uri': 'spotify:artist:6GI52t8N5F02MxU0g5U69P'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',

In [ ]:
album_id = test["album"]["uri"].replace('spotify:album:', '')


In [ ]:
for track_uri in bill_list:
    sleep(1)
    audio_genre_list.append(sp.album(track_uri))
    count += 1
    print(count)

In [ ]:
for track_uri in bill_list:
    sleep(1)
    audio_genre_list.append(sp.album(track_uri))
    count += 1
    print(count)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
type(batch)

list